In [ ]:
import dlt

In [ ]:
display_target = spark.conf.get("display_target")
catalog = spark.conf.get("catalog_name")
raw_data_path = spark.conf.get("raw_data_path")

In [ ]:
@dlt.table
def raw_data():
    return (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("cloudFiles.schemaEvolutionMode", "rescue")
    .option("cloudFiles.inferColumnTypes", "true")
    .load(f"{raw_data_path}")
    )

In [ ]:
@dlt.table
def ingest_data():
    return spark.read.table(f"{catalog}.default.health_silver_demo_01")

In [ ]:
@dlt.table
def joined_data():
    df = (dlt.read("ingest_data").alias("s")
    .join(dlt.read("raw_data").alias("u"), "s.station_id=u.station_id", "inner")
    .selectExpr("s.station_id", "s.name", "s.lat", "s.long", "s.dockcount", "s.landmark", "u._metadata")
    )

    return df

In [ ]:
@dlt.table
def aggragted_data():
    from pyspark.sql.functions import count, sum

    df = (dlt.read("joined_data")
    .groupBy("landmark")
    .agg(count("landmark").alias("count"), sum("dockcount").alias("total"))
    )

    return df